# Tools: Functions

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import time
import requests
import os

In [2]:
load_dotenv()

True

In [3]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [4]:
client = OpenAI()

**Start by defining the functions**

- Location to latitude and longitude using OpenCage
- Get weather using OpenWeather

To get weather, it is required to call the function that gets latitude and longitude

In [5]:
OPENCAGE_API_KEY = os.environ["OPENCAGE_API_KEY"]
OPENWEATHERMAP_API_KEY = os.environ["OPENWEATHERMAP_API_KEY"]

**OpenCage function**

In [6]:
def getLatLong(location):
    base_url = "https://api.opencagedata.com/geocode/v1/json"
    params = {
        "q": location,
        "key": OPENCAGE_API_KEY
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if response.status_code == 200 and data.get("results"):
        lat = data["results"][0]["geometry"]["lat"]
        lon = data["results"][0]["geometry"]["lng"]
        return lat, lon
    else:
        return None

In [7]:
getLatLong("San Francisco, CA")

(37.7790262, -122.419906)

**OpenWeather function**

In [8]:
def getCurrentWeather(location):

    lat, lon = getLatLong(location)
    api_key = OPENWEATHERMAP_API_KEY

    base_url = "https://api.openweathermap.org/data/2.5/weather"
    url = f"{base_url}?lat={lat}&lon={lon}&units=metric&appid={api_key}"

    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        temperature = data["main"]["temp"]
        description = data["weather"][0]["description"]
        return f"The current weather in {location} is {temperature}°C with {description}."
    else:
        return f"Failed to get weather data. Error: {data['message']}"

In [9]:
getCurrentWeather("San Francisco, CA")

'The current weather in San Francisco, CA is 13.51°C with overcast clouds.'

**Describe function that is going to be used by the Assistant**

In [10]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
}

**Execute the function call**

In [11]:
def execute_function_call(function_name,arguments):
    function = available_functions.get(function_name,None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results

**Define tools for the Assistant**

In [12]:
tools = [
    {
        "type": "function",
        "function": {
        "name": "getCurrentWeather",
        "description": "Get the weather in location",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
            "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
        }
        }
    }
]

**Create the Assistant**

In [13]:
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4-1106-preview",
  tools=tools
)

**Create Message and Run**

In [14]:
def create_message_and_run(assistant,query,thread=None):
  if not thread:
    thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=query
  )
  run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
  )
  return run,thread



In [15]:
query = "What's the weather in San Francisco right now?" 

In [16]:
run,thread = create_message_and_run(assistant=assistant,query=query)

**Process function call requirements**

In [17]:
def get_function_details(run):

  print("\nrun.required_action\n",run.required_action)

  function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
  arguments = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
  function_id = run.required_action.submit_tool_outputs.tool_calls[0].id

  print(f"function_name: {function_name} and arguments: {arguments}")

  return function_name, arguments, function_id

**Submit function response**

In [18]:
def submit_tool_outputs(run,thread,function_id,function_response):
    run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
      {
        "tool_call_id": function_id,
        "output": str(function_response),
      }
    ]
    ) 
    return run

**Loop for continuous interaction**

In [19]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print("run status", run.status)

    if run.status=="requires_action":
        function_name, arguments, function_id  = get_function_details(run)
        function_response = execute_function_call(function_name,arguments)
        run = submit_tool_outputs(run,thread,function_id,function_response)
        continue

    if run.status=="completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)

        print("[Write STOP to exit]")
        user_input = input()
        if user_input == "STOP":
          break

        run,thread = create_message_and_run(assistant=assistant,query=user_input,thread=thread)

        continue

run status requires_action

run.required_action
 RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_qwbjpiX8IEDjLzAr7CJFFZMY', function=Function(arguments='{"location":"San Francisco, CA"}', name='getCurrentWeather'), type='function')]), type='submit_tool_outputs')
function_name: getCurrentWeather and arguments: {"location":"San Francisco, CA"}
run status queued
run status queued
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status completed
The weather in San Francisco right now is 13.44°C with overcast clouds.
[Write STOP to exit]


 STOP
